In [13]:
from datasets import SHD_dataloaders, SSC_dataloaders, GSC_dataloaders
from config import Config
from snn_delays import SnnDelays
from snn_delays_daleian import SnnDelays_Dale
from snn_daleian import SNN_Dale
import torch
from snn import SNN
import utils

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n=====> Device = {device} \n\n")


=====> Device = cpu 




In [15]:
config = Config()

if config.model_type == 'snn':
    model = SNN(config).to(device)
if config.model_type == 'snn_delays_dale':
    model = SnnDelays_Dale(config).to(device)
if config.model_type == 'snn_dale':
    model = SNN_Dale(config).to(device)
else:
    model = SnnDelays(config).to(device)

if config.model_type == 'snn_delays_lr0':
    model.round_pos()


print(f"===> Dataset    = {config.dataset}")
print(f"===> Model type = {config.model_type}")
print(f"===> Model size = {utils.count_parameters(model)}\n\n")

je suis là
===> Dataset    = shd
===> Model type = snn_dale
===> Model size = 214016




/Users/alexandrequeant/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [16]:
if config.dataset == 'shd':
    train_loader, valid_loader = SHD_dataloaders(config)
    test_loader = None
elif config.dataset == 'ssc':
    train_loader, valid_loader, test_loader = SSC_dataloaders(config)
elif config.dataset == 'gsc':
    train_loader, valid_loader, test_loader = GSC_dataloaders(config)
else:
    raise Exception(f'dataset {config.dataset} not implemented')

The directory [Datasets/SHD/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [Datasets/SHD/extract] manually, then SpikingJelly will re-extract files from [Datasets/SHD/download].
The directory [Datasets/SHD/duration_10] already exists.
The directory [Datasets/SHD/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [Datasets/SHD/extract] manually, then SpikingJelly will re-extract files from [Datasets/SHD/download].
The directory [Datasets/SHD/duration_10] already exists.


In [17]:
#model.train_model(train_loader, valid_loader, test_loader, device)

In [18]:
# Get one batch from the train_loader
train_iter = iter(train_loader)
first_batch = next(train_iter)

In [19]:
# Assuming the batch is a tuple of (data, labels)
data, labels, _ = first_batch

# Print the first data point and its label
print(data[0], labels[0])

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64) tensor(0)


In [20]:
data.size()

torch.Size([256, 121, 140])

In [21]:
data = data.permute(1,0,2).float().to(device)  #(time, batch, neurons)

In [12]:
data.size()

torch.Size([121, 256, 140])

In [ ]:
output = model.forward(data)

In [ ]:
output

In [ ]:
x = torch.rand(10, 2) > 0.2
x

In [ ]:
x[:, 1]

In [ ]:
model.train_model(train_loader, valid_loader, test_loader, device)

# **Nouvelle classe linéaire**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from DaleLinear import DaleLinear